# ONNX to TF

In [1]:
%cd /workspace/

/workspace


In [2]:
import os

import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['LD_LIBRARY_PATH'] = '${CONDA_PREFIX}/lib:${LD_LIBRARY_PATH}'

# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# from tensorflow.compat.v1 import ConfigProto, InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

image_size = 320

onnx_model_path = f"yolov7-tiny_fp32_{image_size}.onnx"
tf_model_path = f"yolov7-tiny-tf-{image_size}"

onnx_model = onnx.load(onnx_model_path)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(tf_model_path)  # export the model

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

INFO:tensorflow:Assets written to: yolov7-tiny-tf-320/assets


INFO:tensorflow:Assets written to: yolov7-tiny-tf-320/assets


## TF Model Inference

In [3]:
import tensorflow as tf

input_shape = (1, 320, 320, 3)

model = tf.saved_model.load(tf_model_path)
model.trainable = False

input_tensor = tf.random.uniform(input_shape, 0, 255)
out = model(images=input_tensor)
for k in out:
    print(k, out[k].shape)

325 (1, 10, 10, 255)
317 (1, 20, 20, 255)
output (1, 40, 40, 255)


# TF to TFLite

In [4]:
import tensorflow as tf

tflite_model_path = f'yolov7-tiny_fp32_{image_size}.tflite'
# tf_model_path = "yolov7-tiny-tf"

# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
# model = tf.keras.models.load_model(tf_model_path, compile=False)
# func = tf.function(model).get_concrete_function(
#     images=tf.TensorSpec(input_shape, tf.float32)
# )
# converter = tf.lite.TFLiteConverter.from_concrete_functions([func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2023-10-29 00:17:51.751457: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-10-29 00:17:51.751478: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-10-29 00:17:51.752111: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: yolov7-tiny-tf-320
2023-10-29 00:17:51.760157: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-10-29 00:17:51.760178: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: yolov7-tiny-tf-320
2023-10-29 00:17:51.772486: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-10-29 00:17:51.843314: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: yolov7-tiny-tf-320
2023-10-29 00:17:51.925377: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1

Estimated count of arithmetic ops: 3.438 G  ops, equivalently 1.719 G  MACs


## Set model metadata

In [5]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

LABLE_FILE = os.path.basename('label.txt')

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = tflite_model_path.split('.')[0]

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = ("image")
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = LABLE_FILE
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_class_meta.associatedFiles = [label_file]

output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT()
)

output_identity_meta = _metadata_fb.TensorMetadataT()
output_identity_meta.name = "Identity"
output_identity_1_meta = _metadata_fb.TensorMetadataT()
output_identity_1_meta.name = "Identity_1"
output_identity_2_meta = _metadata_fb.TensorMetadataT()
output_identity_2_meta.name = "Identity_2"

# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
# subgraph.outputTensorMetadata = [output_location_meta, output_score_meta, output_class_meta, output_number_meta]
subgraph.outputTensorMetadata = [output_identity_meta, output_identity_1_meta, output_identity_2_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

populator = _metadata.MetadataPopulator.with_model_file(tflite_model_path)
populator.load_associated_files([LABLE_FILE])
populator.load_metadata_buffer(metadata_buf)
populator.populate()

/home/hanyong/.local/lib/python3.8/site-packages/tensorflow_lite_support/metadata/python/metadata.py:395: UserWarning: File, 'label.txt', does not exist in the metadata. But packing it to tflite model is still allowed.
  warnings.warn(


## TFLite model inference

In [6]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
for i in range(len(output_details)):
    print(output_details[i]['index'], interpreter.get_tensor(output_details[i]['index']).shape)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


405 (1, 10, 10, 255)
404 (1, 20, 20, 255)
403 (1, 40, 40, 255)
